# Data Frame Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [220]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt


spark = SparkSession \
    .builder \
    .appName("Quiz1") \
    .getOrCreate()


path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)


In [221]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id "" (empty string) NOT visit?

In [222]:
user_log.filter(user_log.userId=="").select("page").dropDuplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



In [223]:
x = user_log.filter(user_log.userId=="").select("page").dropDuplicates().collect()

In [224]:
user_log.select("page").dropDuplicates().show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|           Login|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



In [225]:
y =user_log.select("page").dropDuplicates().collect()

In [226]:
np.setdiff1d(np.array(y),np.array(x))

array(['Downgrade', 'Error', 'Logout', 'NextSong', 'Save Settings',
       'Settings', 'Submit Downgrade', 'Submit Upgrade', 'Upgrade'], 
      dtype='<U16')

In [227]:
(user_log.select("page").dropDuplicates().subtract(user_log.filter(user_log.userId=="").select("page").dropDuplicates())).collect()

[Row(page='Submit Downgrade'),
 Row(page='Downgrade'),
 Row(page='Logout'),
 Row(page='Save Settings'),
 Row(page='Settings'),
 Row(page='NextSong'),
 Row(page='Upgrade'),
 Row(page='Error'),
 Row(page='Submit Upgrade')]

# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [228]:
user_log.filter(user_log.userId=="").show()
#

+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|artist|      auth|firstName|gender|itemInSession|lastName|length|level|location|method| page|registration|sessionId|song|status|           ts|userAgent|userId|
+------+----------+---------+------+-------------+--------+------+-----+--------+------+-----+------------+---------+----+------+-------------+---------+------+
|  null|Logged Out|     null|  null|            0|    null|  null| free|    null|   PUT|Login|        null|     5598|null|   307|1513721196284|     null|      |
|  null|Logged Out|     null|  null|           26|    null|  null| paid|    null|   GET| Home|        null|      428|null|   200|1513721274284|     null|      |
|  null|Logged Out|     null|  null|            5|    null|  null| free|    null|   GET| Home|        null|     2941|null|   200|1513722009284|     null|      |
|  null|Logged Out|     null|  nul

In [229]:
user_log.filter(user_log.userId=="").select("song").dropDuplicates().show()

+----+
|song|
+----+
|null|
+----+



In [230]:
user_log.filter(user_log.userId=="").select("level").dropDuplicates().show()

+-----+
|level|
+-----+
| free|
| paid|
+-----+



In [231]:
user_log.filter(user_log.userId=="").select("status").dropDuplicates().show()

+------+
|status|
+------+
|   307|
|   200|
+------+



In [232]:
user_log.filter(user_log.userId=="").select("userAgent").dropDuplicates().show()

+---------+
|userAgent|
+---------+
|     null|
+---------+



unregistered users, or ones haven't listened to a song yet

# Question 3

How many female users do we have in the data set?

In [233]:
user_log_valid = user_log.dropna(how = "any", subset = ["userId", "sessionId"])
user_log_valid = user_log_valid.filter(user_log_valid["userId"] != "")
user_log_valid.filter(user_log_valid.gender == "F").select("userId").dropDuplicates().count()
#user_log.select("gender").show()

462

In [234]:
user_log_valid.select(["userId","gender"]).dropDuplicates().groupBy(user_log_valid.gender).count().show()

+------+-----+
|gender|count|
+------+-----+
|     F|  462|
|     M|  501|
+------+-----+



# Question 4

How many songs were played from the most played artist?

In [235]:
user_log_valid.filter(user_log_valid.page == 'NextSong').select(["artist", "song", "userId"]).groupBy("artist").count().sort("count", ascending=False).show(5)

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
+--------------------+-----+
only showing top 5 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [277]:
Home_visit = udf(lambda x: 1 if x == "Home" else 0, IntegerType())
user_log_valid = user_log_valid.withColumn("Home_visit", Home_visit("page"))


In [278]:
user_log_valid.select("Home_visit").show()

+----------+
|Home_visit|
+----------+
|         0|
|         0|
|         0|
|         0|
|         1|
|         0|
|         0|
|         0|
|         1|
|         1|
|         0|
|         0|
|         0|
|         0|
|         0|
|         0|
|         1|
|         0|
|         0|
|         0|
+----------+
only showing top 20 rows



In [279]:
user_log_valid.select(["userId", "song", "page", "Home_visit"]).show(10)

+------+--------------------+--------+----------+
|userId|                song|    page|Home_visit|
+------+--------------------+--------+----------+
|  1046|Christmas Tears W...|NextSong|         0|
|  1000|       Cheryl Tweedy|NextSong|         0|
|  2219|Good Girls Go Bad...|NextSong|         0|
|  2373| Don't See The Point|NextSong|         0|
|  1747|                null|    Home|         1|
|  1747|                null|Settings|         0|
|  1162|        Smoke Buddah|NextSong|         0|
|  1061|           On My Own|NextSong|         0|
|   748|                null|    Home|         1|
|   597|                null|    Home|         1|
+------+--------------------+--------+----------+
only showing top 10 rows



In [280]:
user_log_valid.select(["userId", "song", "page", "Home_visit"]).groupby("userid").count().show(10)

+------+-----+
|userid|count|
+------+-----+
|  2904|    1|
|   691|    4|
|  2294|   62|
|  2162|   39|
|  1436|    2|
|  2088|   14|
|  2275|    5|
|  2756|    8|
|   800|    3|
|  1394|   28|
+------+-----+
only showing top 10 rows



In [281]:
user_log_valid.select(["userId", "ts", "song", "page", "Home_visit"]).orderBy(["userid","ts"]).show(30)

+------+-------------+--------------------+--------+----------+
|userId|           ts|                song|    page|Home_visit|
+------+-------------+--------------------+--------+----------+
|    10|1513790894284|             Secrets|NextSong|         0|
|    10|1513828388284|             Overdue|NextSong|         0|
|   100|1513750214284|                1972|NextSong|         0|
|   100|1513750442284|             Secrets|NextSong|         0|
|   100|1513775431284|                null|    Home|         1|
|   100|1513775556284|Don't It Make My ...|NextSong|         0|
|   100|1513775710284|Clouds (Of Color ...|NextSong|         0|
|   100|1513776194284|                null|    Home|         1|
|   100|1513776308284|                0010|NextSong|         0|
|   100|1513839673284|                null|    Home|         1|
|  1000|1513720878284|       Cheryl Tweedy|NextSong|         0|
|  1003|1513749501284|                null|    Home|         1|
|  1003|1513749516284|         The Hipst

In [282]:
from pyspark.sql import Window
windowval = Window.partitionBy(["userId"]).orderBy(desc('ts')).rangeBetween(Window.unboundedPreceding, 0)

# none worked when "Home" 0
#rangeBetween(Window.unboundedPreceding, 0)
#windowval2 = Window.partitionBy(["userId", "Home_visit"]).orderBy(["ts"]).rangeBetween(0, Window.unboundedFollowing)
#windowval3 = Window.partitionBy(["userId", "Home_visit"]).orderBy(["ts"]).rowsBetween(Window.unboundedPreceding, Window.currentRow)


In [283]:
from pyspark.sql.functions import avg, col
from pyspark.sql.functions import count
from pyspark.sql.functions import row_number

user_log_valid = user_log_valid.withColumn("period", Fsum("Home_visit").over(windowval))


In [285]:
user_log_valid.select(["userId", "page", "Home_visit", "period"]).show(70)

+------+---------+----------+------+
|userId|     page|Home_visit|period|
+------+---------+----------+------+
|  1436| NextSong|         0|     0|
|  1436| NextSong|         0|     0|
|  2088|     Home|         1|     1|
|  2088| NextSong|         0|     1|
|  2088| NextSong|         0|     1|
|  2088| NextSong|         0|     1|
|  2088| NextSong|         0|     1|
|  2088| NextSong|         0|     1|
|  2088| NextSong|         0|     1|
|  2088| NextSong|         0|     1|
|  2088| NextSong|         0|     1|
|  2088| NextSong|         0|     1|
|  2088| NextSong|         0|     1|
|  2088| NextSong|         0|     1|
|  2088| NextSong|         0|     1|
|  2088| NextSong|         0|     1|
|  2162|Downgrade|         0|     0|
|  2162| NextSong|         0|     0|
|  2162| NextSong|         0|     0|
|  2162| NextSong|         0|     0|
|  2162| NextSong|         0|     0|
|  2162| NextSong|         0|     0|
|  2162| NextSong|         0|     0|
|  2162| NextSong|         0|     0|
|

In [286]:
x = user_log_valid.filter((user_log_valid.page == 'NextSong') | (user_log_valid.page == 'Home')) \
    .select('userID', 'page', 'ts', 'Home_visit', 'period')

In [287]:
x.show(50)

+------+--------+-------------+----------+------+
|userID|    page|           ts|Home_visit|period|
+------+--------+-------------+----------+------+
|  1436|NextSong|1513783259284|         0|     0|
|  1436|NextSong|1513782858284|         0|     0|
|  2088|    Home|1513805972284|         1|     1|
|  2088|NextSong|1513805859284|         0|     1|
|  2088|NextSong|1513805494284|         0|     1|
|  2088|NextSong|1513805065284|         0|     1|
|  2088|NextSong|1513804786284|         0|     1|
|  2088|NextSong|1513804555284|         0|     1|
|  2088|NextSong|1513804196284|         0|     1|
|  2088|NextSong|1513803967284|         0|     1|
|  2088|NextSong|1513803820284|         0|     1|
|  2088|NextSong|1513803651284|         0|     1|
|  2088|NextSong|1513803413284|         0|     1|
|  2088|NextSong|1513803254284|         0|     1|
|  2088|NextSong|1513803057284|         0|     1|
|  2088|NextSong|1513802824284|         0|     1|
|  2162|NextSong|1513781246284|         0|     0|


In [288]:
x.filter((x.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}).show(50)

+------+------+-------------+
|userID|period|count(period)|
+------+------+-------------+
|  1436|     0|            2|
|  2088|     1|           13|
|  2162|     0|           19|
|  2162|     2|           15|
|  2294|     0|            4|
|  2294|     1|           17|
|  2294|     2|            3|
|  2294|     3|           16|
|  2294|     4|            4|
|  2294|     5|           11|
|  2904|     0|            1|
|   691|     1|            3|
|  1394|     0|            9|
|  1394|     1|           17|
|  2275|     0|            3|
|  2756|     0|            4|
|  2756|     2|            1|
|   451|     0|            1|
|   451|     1|            1|
|   800|     1|            2|
|  2696|     0|            1|
|   870|     0|            2|
|   926|     0|            2|
|   926|     1|            2|
|  1903|     1|            5|
|   591|     0|            1|
|     7|     0|            2|
|   307|     0|            1|
|   574|     1|            4|
|   613|     0|            1|
|   613|  

In [290]:
x.filter((x.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'})\
    .agg({'count(period)':'avg'}).show(50)

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+



The only way to do this is create an extra column of period to add extra category which is number of time user been in and out of home page. this is only possible if visting home is set to 1 and else is 0. otherwise this become like a counter not a category, and none of the bound conditions don't work if set "Home" to 0. Then for each user and each category do the counting, within the "Next song" filter, then get average of these sums.